In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plot
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df1 = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

t = np.array(df.iloc[6,1:])
np.reshape(t,(28,28))
print(t,df['label'][6])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('label',axis=1), df['label'], test_size=0.2, random_state=42)

In [ ]:
X_train = np.array(X_train)
X_train = np.array_split(X_train, len(X_train))
for t in range(len(X_train)):
    X_train[t] = np.reshape(X_train[t], (28,28))
X_train = np.expand_dims(X_train,-1)
np.shape(X_train)

In [ ]:
X_test = np.array(X_test)
X_test = np.array_split(X_test, len(X_test))
for t in range(len(X_test)):
    X_test[t] = np.reshape(X_test[t], (28,28))
X_test = np.expand_dims(X_test,-1)
np.shape(X_test)

In [ ]:
tgen = ImageDataGenerator( rescale=1./255,
      rotation_range=10,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.01,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest'
                                  )
vgen = ImageDataGenerator( rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.01,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest'
                                  )

batchSize = 128
train_generator = tgen.flow(X_train,y_train,batch_size=batchSize)

val_generator = vgen.flow(X_test, y_test,batch_size=batchSize)




In [ ]:
ad  = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
ad2 =tf.keras.optimizers.Adam(lr =0.001)

In [ ]:

    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape = (28,28,1), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(10, activation = 'softmax')
])
    
    model.compile(optimizer = ad, metrics = ['accuracy'], loss = 'sparse_categorical_crossentropy')
    model.summary()

In [ ]:
call_back = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.25,
                                              verbose=2,
                                              patience=2,
                                              min_lr=0.000001)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('val_accuracy')>0.99):
                print("Reached 99% val_accuracy so cancelling training!")
                self.model.stop_training = True
                

callbacks = myCallback()

In [ ]:
history = model.fit_generator(train_generator,verbose=1,epochs =100,validation_data = val_generator, callbacks = [callbacks,reduce_lr])

In [ ]:
import matplotlib.image as mpimg
import random 
k = np.array(X_train)
t = k[random.randint(0,100)]
t = np.reshape(t,(28,28))
#t = np.expand_dims(t,-1)
g = plot.imshow(t,cmap = 'gray')

In [ ]:
from keras import models
layers_outputs = [layer.output for layer in model.layers[:20]]
activation_model = models.Model(inputs=model.input, outputs=layers_outputs) 

In [ ]:
X_train[9].shape

In [ ]:
k=random.randint(0,100)
img_tensor = X_train[k]
img_tensor = np.expand_dims(img_tensor,0)
activations = activation_model.predict(img_tensor) 
n =17
t = np.reshape(X_train[k],(28,28))
#fig, axs = plot.subplots(n+1,1)
plot.matshow(t, cmap='gray')
for h in range(n):
    first_layer_activation = activations[h]
    print(first_layer_activation.shape)
    plot.matshow(first_layer_activation[0, :, :, 4], cmap='gray')

In [ ]:
print(history.history)


In [ ]:
g = sns.lineplot(history.epoch[6:],history.history['accuracy'][6:],label = 'accuracy')
g.set(xlabel = 'epoc',ylabel  = 'acc')
sns.lineplot(history.epoch[6:],history.history['val_accuracy'][6:], label = 'val_accuracy')


In [ ]:
g = sns.lineplot(history.epoch[6:],history.history['loss'][6:],label = 'loss')
g.set(xlabel = 'epoc',ylabel  = 'loss')
sns.lineplot(history.epoch[6:],history.history['val_loss'][6:], label = 'val_loss')


In [ ]:
df1 = df1/255.0
X_test = df1
X_test = np.array(X_test)
X_test = np.array_split(X_test, len(X_test))
for t in range(len(X_test)):
    X_test[t] = np.reshape(X_test[t], (28,28))
X_test = np.expand_dims(X_test,-1)

k = model.predict(X_test)

k = np.argmax(k,axis = 1)

In [ ]:
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
np.size(k)
sample['Label'] = k
sample.to_csv('sol.csv',index = False)

In [ ]:
sample